In [1]:
import pandas as pd
import matplotlib as plt

In [2]:
df = pd.read_csv('dataUser2.csv')

In [3]:
df[df['ID_INPUT'] == 4]['VALUE'].value_counts()

chrome.exe                   235
explorer.exe                 175
VsDebugConsole.exe           100
Messenger.exe                 84
DB Browser for SQLite.exe     72
Unable To Open Process        46
devenv.exe                    46
Teams.exe                     43
ApplicationFrameHost.exe      30
SearchHost.exe                26
Zoom.exe                      23
cmd.exe                       21
msedge.exe                    20
ShellExperienceHost.exe       12
Spotify.exe                    8
msteams.exe                    6
MoNotificationUx.exe           3
Docker Desktop.exe             3
CredentialUIBroker.exe         2
msiexec.exe                    2
Acrobat.exe                    2
Git-2.39.1-64-bit.tmp          1
python-3.11.1-amd64.exe        1
OneDrive.exe                   1
winzip64.exe                   1
ScreenClippingHost.exe         1
PickerHost.exe                 1
rundll32.exe                   1
conhost.exe                    1
LockApp.exe                    1
Widgets.ex

In [10]:
def predict_HMM(df):
    
    def get_data(df):
            
        return df[df['ID_INPUT'] == 4].drop(columns = ['Unnamed: 0']).reset_index(drop = True)
    
    #return get_data(df)
    

In [13]:
predict_HMM(df)['VALUE'].value_counts()

chrome.exe                   235
explorer.exe                 175
VsDebugConsole.exe           100
Messenger.exe                 84
DB Browser for SQLite.exe     72
Unable To Open Process        46
devenv.exe                    46
Teams.exe                     43
ApplicationFrameHost.exe      30
SearchHost.exe                26
Zoom.exe                      23
cmd.exe                       21
msedge.exe                    20
ShellExperienceHost.exe       12
Spotify.exe                    8
msteams.exe                    6
MoNotificationUx.exe           3
Docker Desktop.exe             3
CredentialUIBroker.exe         2
msiexec.exe                    2
Acrobat.exe                    2
Git-2.39.1-64-bit.tmp          1
python-3.11.1-amd64.exe        1
OneDrive.exe                   1
winzip64.exe                   1
ScreenClippingHost.exe         1
PickerHost.exe                 1
rundll32.exe                   1
conhost.exe                    1
LockApp.exe                    1
Widgets.ex